In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd
import numpy as np
import gc
import os
import optuna
import sqlite3
import ray
import matplotlib.pyplot as plt
import polars as pl
from optuna.integration import LightGBMPruningCallback
from autogluon.tabular import TabularPredictor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from joblib import Parallel, delayed
from more_itertools import chunked
from functools import reduce


In [2]:
gc.collect()
df_full = pd.read_parquet('./data/l_vm_completa_train.parquet', engine='fastparquet')# Abrir el archivo parquet y cargarlo en un DataFrame data/l_vm_completa_train_pendientes.parquet

In [3]:
# Eliminar de df_full las filas donde la columna A_PREDECIR sea 'N'
df_full = df_full[df_full['A_PREDECIR'] != 'N']
df_full = df_full.drop(columns=['A_PREDECIR'])

In [4]:
# Conservar las siguientes columnas
columns_to_keep = ['PERIODO', 'ANIO', 'MES', 'MES_SIN', 'MES_COS', 'TRIMESTRE', 'ID_CAT1',
       'ID_CAT2', 'ID_CAT3', 'ID_BRAND', 'SKU_SIZE', 'CUSTOMER_ID',
       'PRODUCT_ID', 'PLAN_PRECIOS_CUIDADOS', 'CUST_REQUEST_QTY',
       'CUST_REQUEST_TN', 'TN', 'CLASE', 'CLASE_DELTA',
       'ORDINAL', 'ANTIG_CLIENTE',
       'ANTIG_PRODUCTO', 'CANT_PROD_CLI_PER']
# Filtrar el DataFrame para conservar solo las columnas deseadas 
df_full = df_full[columns_to_keep]
df_full['DIAS_EN_MES'] = pd.to_datetime(df_full['PERIODO'], format='%Y%m').dt.days_in_month

In [5]:
# 1. Ordenar correctamente
df_full = df_full.sort_values(by=['PRODUCT_ID', 'CUSTOMER_ID', 'ORDINAL'], ascending=True)

# 2. Crear los LAGs y DELTAs
for lag in range(1, 36):
    lag_col = f'TN_LAG_{lag:02d}'
    delta_col = f'TN_DELTA_{lag:02d}'

    df_full[lag_col] = df_full.groupby(['PRODUCT_ID', 'CUSTOMER_ID'])['TN'].shift(lag)
    df_full[delta_col] = df_full['TN'] - df_full[lag_col]

# 3. Crear las variaciones porcentuales SIN fragmentar
delta_pct_cols = {}

for lag in range(1, 36):
    lag_col = f'TN_LAG_{lag:02d}'
    delta_col = f'TN_DELTA_{lag:02d}'
    delta_pct_col = f'TN_DELTA_{lag:02d}_PORC'

    delta_pct_cols[delta_pct_col] = np.where(
        df_full[lag_col] == 0,
        np.nan,
        df_full[delta_col] / df_full[lag_col]
    )

# 4. Concatenar todas las columnas nuevas en un solo paso
df_porcentajes = pd.DataFrame(delta_pct_cols, index=df_full.index)
df_full = pd.concat([df_full, df_porcentajes], axis=1)

# 5. (Opcional) Defragmentar para mejorar rendimiento
df_full = df_full.copy()


In [6]:
# Convertir el DataFrame a un DataFrame de Polars
df_full = pl.from_pandas(df_full)

In [ ]:
# 1. Filtrar columnas relevantes
columnas_a_normalizar = [
    col for col in df_full.columns
    if ('TN' in col or 'DELTA' in col or 'CLASE' in col or 'LAG' in col)
    and 'PORC' not in col
]

# 2. Inicializar DataFrame con combinaciones únicas
medias_y_desvios = df_full.select(['PRODUCT_ID', 'CUSTOMER_ID']).unique()

# 3. Calcular medias y desvíos por columna
resultados = []
for col in columnas_a_normalizar:
    if col in df_full.columns:
        resumen = (
            df_full
            .select(['PRODUCT_ID', 'CUSTOMER_ID', col])
            .group_by(['PRODUCT_ID', 'CUSTOMER_ID'])
            .agg([
                pl.col(col).mean().alias(f'{col}_MEDIA'),
                pl.col(col).std().alias(f'{col}_DESVIO')
            ])
        )
        resultados.append(resumen)

# 4. Combinar todos los resultados
medias_y_desvios = reduce(
    lambda df1, df2: df1.join(df2, on=['PRODUCT_ID', 'CUSTOMER_ID'], how='left'), 
    resultados
)
# Convertir los nulos en ceros
medias_y_desvios = medias_y_desvios.fill_null(0)

In [ ]:
# Imprimir las columnas de medias y desvíos
print(medias_y_desvios.columns)

In [ ]:
def normalizar_zscore_con_join(
    df_full: pl.DataFrame,
    medias_y_desvios: pl.DataFrame,
    columnas_a_normalizar: List[str],
    conservar_auxiliares: bool = False,
    epsilon: float = 1e-6,
    clip_value: float = 5.0,
    agregar_clip: bool = False
) -> pl.DataFrame:
    # 1. Armar columnas necesarias
    columnas_media = [f'{col}_MEDIA' for col in columnas_a_normalizar if f'{col}_MEDIA' in medias_y_desvios.columns]
    columnas_desvio = [f'{col}_DESVIO' for col in columnas_a_normalizar if f'{col}_DESVIO' in medias_y_desvios.columns]
    columnas_join = ['PRODUCT_ID', 'CUSTOMER_ID'] + columnas_media + columnas_desvio

    # 2. Join
    df_aux = medias_y_desvios.select(columnas_join)
    df_full = df_full.join(df_aux, on=['PRODUCT_ID', 'CUSTOMER_ID'], how='left')

    # 3. Calcular ZSCOREs
    zscore_exprs = []
    for col in columnas_a_normalizar:
        media_col = f"{col}_MEDIA"
        desvio_col = f"{col}_DESVIO"
        z_col = f"{col}_ZSCORE"
        if media_col in df_full.columns and desvio_col in df_full.columns:
            expr = (
                (pl.col(col) - pl.col(media_col)) /
                (pl.col(desvio_col) + epsilon)
            ).alias(z_col)
            zscore_exprs.append(expr)
            print(f"✅ Normalizando: {col} -> {z_col}")

    df_full = df_full.with_columns(zscore_exprs)

    # 4. Clipping (después de que los zscores existen)
    if agregar_clip:
        clip_exprs = [
            pl.col(f"{col}_ZSCORE").clip(-clip_value, clip_value).alias(f"{col}_ZSCORE_CLIP")
            for col in columnas_a_normalizar
            if f"{col}_ZSCORE" in df_full.columns
        ]
        df_full = df_full.with_columns(clip_exprs)

    # 5. Eliminar auxiliares si no se quieren
    if not conservar_auxiliares:
        df_full = df_full.drop(columnas_media + columnas_desvio)

    return df_full




In [ ]:
df_full = normalizar_zscore_con_join(
    df_full=df_full,
    medias_y_desvios=medias_y_desvios,
    columnas_a_normalizar=columnas_a_normalizar,
    conservar_auxiliares=False,
    epsilon=1e-6,
    clip_value=5.0,
    agregar_clip=True
)

In [ ]:

# Eliminar las columnas originales de df_full que fueron normalizadas
# son las que están en columnas_a_normalizar
df_full = df_full.drop(columnas_a_normalizar)

In [ ]:
# Convertir de nuevo a DataFrame de Pandas
df_full = df_full.to_pandas()

In [ ]:
# --- Cálculo de features por grupo ---
def calcular_pendientes_grupo(group, periodos_list):
    group = group.sort_values(by='PERIODO').copy()
    n = len(group)
    y_series = pd.Series(group['TN_ZSCORE'].values)

    new_cols = {}

    for cant in periodos_list:
        x = np.arange(cant)
        rolling = y_series.rolling(window=cant, min_periods=1)

        # Medidas estadísticas
        mean_vals = rolling.mean().values
        std_vals = rolling.std().values
        median_vals = rolling.median().values
        min_vals = rolling.min().values
        max_vals = rolling.max().values
        ewma_vals = y_series.ewm(span=cant, adjust=False).mean().values

        new_cols[f'TN_MEAN_ZSCORE_{cant}'] = mean_vals
        new_cols[f'TN_STD_ZSCORE_{cant}'] = std_vals
        new_cols[f'TN_MEDIAN_ZSCORE_{str(cant).zfill(2)}'] = median_vals
        new_cols[f'TN_MIN_ZSCORE_{str(cant).zfill(2)}'] = min_vals
        new_cols[f'TN_MAX_ZSCORE_{str(cant).zfill(2)}'] = max_vals
        new_cols[f'TN_EWMA_ZSCORE_{str(cant).zfill(2)}'] = ewma_vals

        # Pendiente de regresión lineal
        if n >= cant:
            y_rolling = np.lib.stride_tricks.sliding_window_view(y_series.values, window_shape=cant)
            X = np.vstack([x, np.ones(cant)]).T
            XTX_inv_XT = np.linalg.pinv(X)
            betas = XTX_inv_XT @ y_rolling.T
            pendientes = np.full(n, np.nan)
            pendientes[cant - 1:] = betas[0]
        else:
            pendientes = np.full(n, np.nan)
        new_cols[f'PENDIENTE_TENDENCIA_ZSCORE_{cant}'] = pendientes

        # Medidas de variabilidad respecto a la media
        abs_diff = np.abs(y_series.values - mean_vals)
        cv_vals = std_vals / np.where(mean_vals == 0, np.nan, mean_vals)

        new_cols[f'TN_ABS_DIFF_MEAN_ZSCORE_{cant}'] = abs_diff
        new_cols[f'TN_CV_ZSCORE_{cant}'] = cv_vals

    df_features = pd.DataFrame(new_cols, index=group.index)
    group = pd.concat([group, df_features], axis=1)
    return group

# --- Procesar un chunk de grupos ---
def procesar_chunk(chunk, periodos_list):
    return pd.concat([calcular_pendientes_grupo(g, periodos_list) for g in chunk], ignore_index=True)

# --- Paralelización eficiente ---
def calcular_pendientes_parallel_optimizado(df, periodos_list, n_jobs=28, chunk_size=100):
    df = df.copy()  # conserva todas las columnas originales
    grupos = [group for _, group in df.groupby(['PRODUCT_ID', 'CUSTOMER_ID'])]
    chunks = list(chunked(grupos, chunk_size))

    resultados = Parallel(n_jobs=n_jobs, backend='loky', verbose=10)(
        delayed(procesar_chunk)(chunk, periodos_list) for chunk in chunks
    )

    df_final = pd.concat(resultados, ignore_index=True)
    return df_final

# --- Script principal ---
if __name__ == "__main__":
    import time
    start = time.time()

    df_full = calcular_pendientes_parallel_optimizado(
        df_full,
        periodos_list=[2, 3, 6, 9, 12, 13, 15, 18],
        n_jobs=28,
        chunk_size=200
    )

    print(f"Tiempo total: {time.time() - start:.2f} segundos")


In [ ]:
# Umbral: proporción máxima permitida (ej. 0.4 = 40%)
umbral_faltantes = 0.8

# Diccionario para almacenar estadísticas
estadisticas_columnas = []

# Recorremos las columnas del DataFrame
for col in df_full.columns:
    total = len(df_full[col])
    nulls = df_full[col].isnull().sum()
    nans = df_full[col].isna().sum()
    infs = np.isinf(df_full[col]).sum()
    
    total_faltantes = nulls + infs  # NaN está incluido en nulls/isna
    
    porcentaje = total_faltantes / total
    
    estadisticas_columnas.append({
        'columna': col,
        'nulls': nulls,
        'NaNs': nans,
        'infs': infs,
        'porcentaje_faltantes': porcentaje
    })

# Convertir a DataFrame
df_faltantes = pd.DataFrame(estadisticas_columnas)

# Filtrar columnas que superen el umbral
columnas_a_eliminar = df_faltantes[df_faltantes['porcentaje_faltantes'] > umbral_faltantes]

# Mostrar resumen
print(f"\nSe eliminarán {len(columnas_a_eliminar)} columnas con más del {umbral_faltantes*100:.0f}% de valores faltantes o infinitos:")
for _, row in columnas_a_eliminar.iterrows():
    print(f"- {row['columna']}: {row['porcentaje_faltantes']*100:.2f}% (nulls={row['nulls']}, infs={row['infs']})")

# Eliminar columnas del DataFrame
df_full = df_full.drop(columns=columnas_a_eliminar['columna'].tolist())

In [ ]:
# Agregar a df_resultado una variable categorica MES_PROBLEMATICO que sea 1 si PERIODO es 201906 o 201908 o 201910, y 0 en caso contrario
df_full['MES_PROBLEMATICO'] = df_full['PERIODO'].apply(lambda x: True if x in [201906, 201908] else False)
df_full['PLAN_PRECIOS_CUIDADOS'] = df_full['PLAN_PRECIOS_CUIDADOS'].map({1 : True, 0: False})

In [ ]:
#Optimizar tipos de datos numéricos
for col in df_full.select_dtypes(include=['int64']).columns:
    df_full[col] = pd.to_numeric(df_full[col], downcast='integer')
for col in df_full.select_dtypes(include=['float64']).columns:
    df_full[col] = pd.to_numeric(df_full[col], downcast='float')
categorical_features = []

In [ ]:
# Guardar el DataFrame resultante en un archivo parquet
df_full.to_parquet('./data/l_vm_completa_normalizada_fe.parquet', engine='fastparquet', index=False)
medias_y_desvios_pd = medias_y_desvios.to_pandas()
medias_y_desvios_pd.to_parquet('./data/medias_y_desvios.parquet', engine='fastparquet', index=False)